# Implementation of GoogLeNet (Inception)

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, optimizers, Input, Model, losses
from tensorflow.keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten, LayerNormalization, concatenate
from tensorflow.keras.utils import plot_model
import tensorflow_datasets as tfds
import pydot

dataset_name = 'caltech101'
ds_info = tfds.builder(dataset_name).info
# 101 classes and 1 background class
num_classes = ds_info.features['label'].num_classes
num_epochs = 100
num_batch_sizes = 64
print("Datasets: ", dataset_name, "Number of Classes: ", num_classes)
#train_ds, test_ds = tfds.load(dataset_name, split=['train','test'])
train_ds, test_ds = tfds.load(dataset_name, split=['test','train'])

def preprocess(x,y):
    x = 2*tf.cast(x, dtype = tf.float32)/255. -1
    y = tf.cast(y, dtype = tf.int32)
    return x, y

def getDb(ds):
    x, y = [], []
    for sample in ds:
        x.append(tf.image.resize(sample["image"], size = [224, 224]))
        y.append(sample["label"])
    db = tf.data.Dataset.from_tensor_slices((x,y))
    print(len(x))
    return db

train_db = getDb(train_ds)
train_db = train_db.shuffle(1000).map(preprocess).batch(num_batch_sizes)
test_db = getDb(test_ds)
test_db = test_db.map(preprocess).batch(num_batch_sizes)

print("Data sets constructed!")
print("Num of Classes:", num_classes)
#print(train_db, test_db)

def getInceptionModule(x, filters_11, filters_113, filters_33, filters_115, filters_55, filters_11mp):
    x_11 = Conv2D(filters = filters_11, kernel_size = [1, 1], strides = 1, activation='relu')(x)
    
    x_113 = Conv2D(filters = filters_113, kernel_size = [1, 1], strides = 1, activation='relu')(x)
    x_33 = Conv2D(filters = filters_33, kernel_size = [3, 3], strides = 1, padding = 'same', activation = 'relu')(x_113)
    
    x_115 = Conv2D(filters = filters_115, kernel_size = [1, 1], strides = 1, activation='relu')(x)
    x_55 = Conv2D(filters = filters_55, kernel_size = [5, 5], strides = 1, padding='same', activation='relu')(x_115)
    
    x_mp = MaxPool2D(pool_size = [3, 3], padding = 'same', strides = 1)(x)
    x_mp11 = Conv2D(filters = filters_11mp, kernel_size = [1,1], strides = 1, activation='relu')(x_mp)
    
    return concatenate([x_11, x_33, x_55, x_mp11]) 

def getGoogleNet():
    googlenet_inputs = Input(shape=[224, 224, 3])
    x = Conv2D(64, kernel_size = [7,7], strides = 2, padding = 'same', activation='relu')(googlenet_inputs)
    x = MaxPool2D(pool_size=[3,3], strides = 2, padding = 'same')(x)
    #x = LayerNormalization(epsilon = 1e-6)(x)

    x = Conv2D(64, kernel_size=[1,1], activation = 'relu')(x)
    x = Conv2D(192, kernel_size=[3,3], padding='same', activation = 'relu')(x)
    x = MaxPool2D(pool_size=[3,3], strides = 2, padding = 'same')(x)
    #x = LayerNormalization(epsilon = 1e-6)(x)

    inc_3a = getInceptionModule(x, 64, 96, 128, 16, 32, 32)
    inc_3b = getInceptionModule(inc_3a, 128, 128, 192, 32, 96, 64)
    x = MaxPool2D(pool_size = [3, 3], strides = 2, padding = 'same')(inc_3b)

    inc_4a = getInceptionModule(x, 192, 96, 208, 16, 48, 64)
    inc_4b = getInceptionModule(inc_4a, 160, 112, 224, 24, 64, 64)
    inc_4c = getInceptionModule(inc_4b, 128, 128, 256, 24, 64, 64)
    inc_4d = getInceptionModule(inc_4c, 112, 144, 288, 32, 64, 64)
    inc_4e = getInceptionModule(inc_4d, 256, 160, 320, 32, 128, 128)
    x = MaxPool2D(pool_size = [3, 3], strides = 2, padding = 'same')(inc_4e)

    inc_5a = getInceptionModule(x, 256, 160, 320, 32, 128, 128)
    inc_5b = getInceptionModule(inc_5a, 384, 192, 384, 48, 128, 128)

    avg_pool_6 = AveragePooling2D(pool_size = [7, 7], padding = 'same')(inc_5b)
    x = Flatten()(avg_pool_6)
    x = Dropout(rate=0.4)(x)
    x = Dense(1000, activation='relu')(x)
    outputs_7 = Dense(num_classes, activation=None)(x)

    outputs_4a = AveragePooling2D(pool_size= [5, 5], strides = [3,3], padding = 'valid')(inc_4a)
    outputs_4a = Conv2D(filters = 128, kernel_size = [1,1], strides = [1,1], padding = 'same', activation='relu')(outputs_4a)
    outputs_4a = Flatten()(outputs_4a)
    outputs_4a = Dense(1024, activation='relu')(outputs_4a)
    outputs_4a = Dropout(0.7)(outputs_4a)
    outputs_4a = Dense(num_classes, activation='softmax')(outputs_4a)

    outputs_4d = AveragePooling2D(pool_size= [5, 5], strides = [3,3], padding = 'valid')(inc_4d)
    outputs_4d = Conv2D(filters = 128, kernel_size = [1,1], strides = [1,1], padding = 'same', activation='relu')(outputs_4d)
    outputs_4d = Flatten()(outputs_4d)
    outputs_4d = Dense(1024, activation='relu')(outputs_4d)
    outputs_4d = Dropout(0.7)(outputs_4d)
    outputs_4d = Dense(num_classes, activation='softmax')(outputs_4d)
    #googlenet_model = Model(inputs = googlenet_inputs, outputs = [outputs_7, outputs_4a, outputs_4d])
    googlenet_outputs = [outputs_7, outputs_4a, outputs_4d]
    print(googlenet_outputs)
    googlenet_model = Model(inputs = googlenet_inputs, outputs = googlenet_outputs)
    
    return googlenet_model

googlenet = getGoogleNet()
googlenet.compile(optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.9),
                  loss = losses.SparseCategoricalCrossentropy(from_logits = True), 
                  metrics = ['accuracy'])
#googlenet.summary()
#plot_model(googlenet, to_file='googlenet_model.png', show_shapes=True, show_layer_names=True)

history = googlenet.fit(train_db, batch_size = num_batch_sizes, epochs = num_epochs)
test_scores = googlenet.evaluate(test_db, verbose=2)

print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Datasets:  caltech101 Number of Classes:  102
6084
3060
Data sets constructed!
Num of Classes: 102
[<tf.Tensor 'dense_1/Identity:0' shape=(None, 102) dtype=float32>, <tf.Tensor 'dense_3/Identity:0' shape=(None, 102) dtype=float32>, <tf.Tensor 'dense_5/Identity:0' shape=(None, 102) dtype=float32>]
Epoch 1/100
96/96 [==============================] - 16s 162ms/step - loss: 13.2932 - dense_1_loss: 4.0445 - dense_3_loss: 4.6244 - dense_5_loss: 4.6243 - dense_1_accuracy: 0.1269 - dense_3_accuracy: 0.0174 - dense_5_accuracy: 0.0324
Epoch 2/100
96/96 [==============================] - 15s 153ms/step - loss: 12.7919 - dense_1_loss: 3.6745 - dense_3_loss: 4.5469 - dense_5_loss: 4.5706 - dense_1_accuracy: 0.1410 - dense_3_accuracy: 0.1213 - dense_5_accuracy: 0.1228
Epoch 3/100
96/96 [==============================] - 15s 153ms/step - loss: 12.5914 - dense_1_loss: 3.6443 - dense_3_loss: 4.4343 - dense_5_loss: 4.5128 - dense_1_accuracy: 0.1637 - dense_3_accuracy: 0.2094 - dense_5_accuracy: 0.1262


96/96 [==============================] - 15s 155ms/step - loss: 8.7731 - dense_1_loss: 0.1374 - dense_3_loss: 4.3190 - dense_5_loss: 4.3166 - dense_1_accuracy: 0.9586 - dense_3_accuracy: 0.3208 - dense_5_accuracy: 0.3233
Epoch 70/100
96/96 [==============================] - 15s 155ms/step - loss: 8.7396 - dense_1_loss: 0.1056 - dense_3_loss: 4.3173 - dense_5_loss: 4.3167 - dense_1_accuracy: 0.9663 - dense_3_accuracy: 0.3225 - dense_5_accuracy: 0.3231
Epoch 71/100
96/96 [==============================] - 15s 155ms/step - loss: 8.7737 - dense_1_loss: 0.1365 - dense_3_loss: 4.3190 - dense_5_loss: 4.3183 - dense_1_accuracy: 0.9584 - dense_3_accuracy: 0.3235 - dense_5_accuracy: 0.3241
Epoch 72/100
96/96 [==============================] - 15s 155ms/step - loss: 8.7996 - dense_1_loss: 0.1602 - dense_3_loss: 4.3206 - dense_5_loss: 4.3188 - dense_1_accuracy: 0.9545 - dense_3_accuracy: 0.3218 - dense_5_accuracy: 0.3236
Epoch 73/100
96/96 [==============================] - 15s 155ms/step - loss: 

In [ ]:
import tensorflow as tf

print